In [1]:
import os
os.environ["KERAS_BACKEND"] ='tensorflow'
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm.auto import tqdm
tqdm.pandas()
import keras 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from keras import layers, Model, Input
from keras_hub.src.models.llama.llama_decoder import LlamaTransformerDecoder
from keras_hub.src.models.llama.llama_layernorm import LlamaLayerNorm

/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-04 06:43:20.113967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767530600.306717   62293 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767530600.376601   62293 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767530600.727366   62293 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more 

# Data Loading

In [2]:
import h5py
file_path = "./data/SabDab-RELAAI/SabDab_RELAAI_esmt36_Full.h5"
load_seq = {}

with h5py.File(file_path, "r") as hf:
    for seq in tqdm(hf.keys()):
        load_seq[seq] = hf[seq][:]

100%|██████████| 1692/1692 [00:37<00:00, 44.73it/s]


In [3]:
df = pd.read_csv("./data/SabDab-RELAAI/AbAg_record_level_balanced_dataset.csv")
df.head()

,ab_seq,ag_seq,Hseq,Lseq,Ab_cluster,Ag_cluster,label
0,EVQLVESGGGLVQPGGSLRLSCAASGYTFTSYWLHWVRQAPGKGLE...,MKYQLPNFTAETPIQNVILHEHHIFLGATNYIYVLNEEDLQKVAEY...,EVQLVESGGGLVQPGGSLRLSCAASGYTFTSYWLHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCKSSQSLLYTSSQKNYLAWYQQKP...,147,0,1
1,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,VMDFLFEKWKLYGDQCHHNLSLLPPPTELVCNRTFDKYSCWPDTPA...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,66,392,1
2,QVQLQQSGAELARPGASVKLSCKASGYTFTDYYINWMKQRTGQGLE...,REHWATRLGLILAMAGNAVGLGNFLRFPVQAAENGGGAFMIPYIIA...,QVQLQQSGAELARPGASVKLSCKASGYTFTDYYINWMKQRTGQGLE...,DIVLTQSPASLAVSLGQRATISCKASQSVDYDGDSYMNWYQQKPGQ...,148,1,1
3,QVQLQQPGAELVRPGASVKLSCKASGYTLTTYWMNWFKQRPDQGLE...,STATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNN...,QVQLQQPGAELVRPGASVKLSCKASGYTLTTYWMNWFKQRPDQGLE...,DVVMTQTPLSLPVSLGDQASISCRSSQTLVHSNGNTYLHWYLQKPG...,95,2,1
4,QVQLKESGPGLVAPSQSLSITCTVSGFLLISNGVHWVRQPPGKGLE...,STATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNN...,QVQLKESGPGLVAPSQSLSITCTVSGFLLISNGVHWVRQPPGKGLE...,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,195,2,1


In [4]:
df['label'].value_counts()

label
1    3782
0    3782
Name: count, dtype: int64

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

def process_sequence_tf(x_emb, max_len=512, pad_value=0.0):
    seq_len = x_emb.shape[0]
    
    if seq_len > max_len:
        # truncate
        x_emb = tf.convert_to_tensor(x_emb[:max_len])
        mask = tf.ones([max_len], dtype=tf.float32)
    else:
        # pad
        pad_len = max_len - seq_len
        paddings = [[0, pad_len], [0, 0]]
        x_emb = tf.pad(x_emb, paddings, constant_values=pad_value)
        mask = tf.pad(tf.ones([seq_len], dtype=tf.float32), [[0, pad_len]], constant_values=0.0)
    
    return x_emb, mask

# -----------------------------
# Keras Sequence Loader
# -----------------------------
class DataSequenceLoader(Sequence):
    def __init__(self, df, batch_size=32, shuffle=True, max_len=512, pad_value=0.0):
        self.x1_emb = df["ab_seq"].values
        self.x2_emb = df["ag_seq"].values
        self.labels = df["label"].values
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.max_len = max_len
        self.pad_value = pad_value
        self.indices = np.arange(len(df))
        if shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, idx):
        batch_idx = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        x1_list, x2_list, m1_list, m2_list = [], [], [], []
        labels_list = []

        for i in batch_idx:
            emb1 = load_seq[self.x1_emb[i]].reshape(-1, 2560)
            emb2 = load_seq[self.x2_emb[i]].reshape(-1, 2560)

            # ---- process embeddings + mask ----
            x1_pad, mask1 = process_sequence_tf(emb1, max_len=self.max_len, pad_value=self.pad_value)
            x2_pad, mask2 = process_sequence_tf(emb2, max_len=self.max_len, pad_value=self.pad_value)

            x1_list.append(x1_pad.numpy())
            x2_list.append(x2_pad.numpy())
            m1_list.append(mask1.numpy())
            m2_list.append(mask2.numpy())
            labels_list.append(self.labels[i])

        # Convert lists to arrays for batch
        x1_batch = np.stack(x1_list, axis=0)
        x2_batch = np.stack(x2_list, axis=0)
        m1_batch = np.stack(m1_list, axis=0)
        m2_batch = np.stack(m2_list, axis=0)
        labels_batch = np.array(labels_list)

        return (x1_batch, x2_batch, m1_batch, m2_batch), labels_batch


In [6]:
# loader  = DataSequenceLoader(df,batch_size=4,shuffle =True)
# for x in loader:
#     break

In [7]:
# -------------------------------------------------------------------
# Hybrid Pooling Layer (Max + Avg)
# -------------------------------------------------------------------
class HybridPooling(layers.Layer):
    def call(self, x):
        max_pooled = keras.ops.max(x, axis=1)
        avg_pooled = keras.ops.mean(x, axis=1)
        return keras.ops.concatenate([max_pooled, avg_pooled], axis=-1)


# -------------------------------------------------------------------
# Conv Block (Conv1D → ReLU → Dropout → MaxPool)
# -------------------------------------------------------------------
def conv_block(x, filters =  100, kernel_sz =20, stride =10, dropout = 0.5):
    x = layers.Conv1D(filters, kernel_sz, strides=stride)(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.MaxPooling1D(pool_size=3, strides=1, padding="same")(x)
    return x


# -------------------------------------------------------------------
# Cross-attention (Query=X1, Key=X2, Value=X2)
# -------------------------------------------------------------------
def cross_attention_block(query, key, value, mask, num_heads =4, key_dim =32):
    attn = layers.MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=key_dim,
        dropout=0.0,
        kernel_initializer="glorot_uniform",
        use_bias=True,
        # flash_attention=None,  # if GPU supports
    )(query, key, value, key_mask=mask, value_mask=mask)
    return attn


# -------------------------------------------------------------------
# LLaMA Self-Attention Block
# -------------------------------------------------------------------
def llama_self_attention(x, mask, hidden_dim =100, num_heads = 4):
    # single LLaMA decoder layer
    llama = LlamaTransformerDecoder(
        intermediate_dim=hidden_dim * 4,
        # num_heads=num_heads,
        num_query_heads=8,
        num_key_value_heads=2,
        dropout=0.0,
        layer_norm_epsilon=1e-5,
        activation="silu"
    )
    return llama(x, decoder_padding_mask=mask)

# -------------------------------------------------------------------
# Functional API Model (PyTorch → Keras Conversion)
# -------------------------------------------------------------------
def build_model(
    input_dim=2560,
    conv_out=100,
    kernel_sz=20,
    stride=10,
    heads=4,
    d_dim=32,
    drop_pool=0.5,
    drop_linear=0.3
):
    # Inputs
    inp1 = Input((None, 2560))
    inp2 = Input((None, 2560))
    mask1 = Input((None, ))
    mask2 = Input((None, ))

    # ---------------------------------------------------
    # 1) Convolutional features
    # ---------------------------------------------------
    # p1 = conv_block(inp1, conv_out, kernel_sz, stride, drop_pool)
    # p2 = conv_block(inp2, conv_out, kernel_sz, stride, drop_pool)
    x_dim = 384
    p1 = layers.Dense(x_dim, use_bias=False,name='stem1')(inp1)
    p1 = layers.BatchNormalization(momentum=0.95,name='bn1')(p1)
    
    p2 = layers.Dense(x_dim, use_bias=False,name='stem2')(inp2)
    p2 = layers.BatchNormalization(momentum=0.95,name='bn2')(p2)
    
    # ---------------------------------------------------
    # 2) Self Attention using LLaMA blocks
    # ---------------------------------------------------
    s1 = llama_self_attention(p1, mask1, conv_out, heads)
    s2 = llama_self_attention(p2, mask2, conv_out, heads)

    # ---------------------------------------------------
    # 3) Cross Attention (1→2 and 2→1)
    # ---------------------------------------------------
    c1 = cross_attention_block(p1, p2, p2, mask2, heads, d_dim)
    c2 = cross_attention_block(p2, p1, p1, mask1, heads, d_dim)

    # Add residual (same as PyTorch + skip)
    sc1 = layers.Add()([s1, c1])
    sc2 = layers.Add()([s2, c2])

    sc1 = layers.Dropout(drop_pool)(sc1)
    sc2 = layers.Dropout(drop_pool)(sc2)

    # ---------------------------------------------------
    # 4) Hybrid Pooling (max + mean)
    # ---------------------------------------------------
    h1 = HybridPooling()(sc1)
    h2 = HybridPooling()(sc2)

    merged = layers.Concatenate()([h1, h2])

    # ---------------------------------------------------
    # 5) MLP Head
    # ---------------------------------------------------
    x = layers.Dense(256, activation="relu")(merged)
    x = layers.Dropout(drop_linear)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(drop_linear)(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    return Model(inputs=[inp1, inp2, mask1, mask2], outputs=out)




In [8]:
import os
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, average_precision_score, confusion_matrix
)
import wandb
from wandb.integration.keras import WandbMetricsLogger

In [9]:
# =========================================================
# DEFAULT CONFIG (Logged Automatically to W&B)
# =========================================================
TIMESTAMP = time.strftime("%Y%m%d-%H%M%S")

CONFIG = {
    "batch_size": 64,
    "n_splits": 5,
    "epochs": 70,
    "learning_rate": 5e-5,
    "optimizer": "Adam",
    "loss": "binary_crossentropy",
    "architecture": "ESM_T36",
    "dataset": "SabDab2",
    "task": "Prot-Prot Classification",
    "random_state": 42,
    "max_len":512,
}

PROJECT_NAME = f"{CONFIG['dataset']}-RLEAAI-{CONFIG['architecture']}-{TIMESTAMP}"
OUT_PATH = os.path.join("weights",PROJECT_NAME)
os.makedirs(os.path.join(OUT_PATH, "logs"), exist_ok=True)
os.makedirs(os.path.join(OUT_PATH, "weights"), exist_ok=True)
OUT_PATH

'weights/SabDab2-RLEAAI-ESM_T36-20260104-064451'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, matthews_corrcoef, confusion_matrix,
    average_precision_score
)
CONFIG['n_splits'] =5
# =============================================================
#  K-FOLD CROSS VALIDATION SETTINGS
# =============================================================
skf = StratifiedKFold(n_splits=CONFIG['n_splits'], shuffle=True, random_state=42)

all_metrics = []

# =============================================================
#  MAIN LOOP
# =============================================================
for fold,(train_idx, valid_idx) in enumerate(skf.split(df, df["label"]),1):
    print(f"\n==========================")
    print(f" Fold {fold} / {CONFIG['n_splits']}")
    print(f"==========================")
    run = wandb.init(
            project=PROJECT_NAME,
            name=f"fold_{fold}",
            group="KFold-CV",
            config={**CONFIG, "fold": fold},
            reinit=True
        )
    train_df = df.iloc[train_idx].reset_index(drop=True)
    valid_df = df.iloc[valid_idx].reset_index(drop=True)

    # ---------------------------------------
    # Loaders
    # ---------------------------------------
    train_loader = DataSequenceLoader(
        train_df, 
        batch_size=CONFIG["batch_size"],
        max_len = CONFIG['max_len'], 
        shuffle=True)
    valid_loader = DataSequenceLoader(
        valid_df, 
        batch_size=CONFIG["batch_size"],
        max_len = CONFIG['max_len'], 
        shuffle=False)

    # ---------------------------------------
    # Build a FRESH MODEL per fold
    # ---------------------------------------
    model = build_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(CONFIG["learning_rate"]),
        loss=CONFIG["loss"],
        metrics=[
            "accuracy",
            tf.keras.metrics.AUC(name="auc")
        ]
    )
    # -----------------------------------------------------
    # Callbacks
    # -----------------------------------------------------
    tb_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(OUT_PATH, "logs", f"fold_{fold}")
    )

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(
            OUT_PATH, "weights",
            f"weights_fold{fold}-best.weights.h5"
        ),
        save_weights_only=True,
        save_best_only=True,
        monitor="val_loss"
    )
    # ---------------------------------------
    # Train
    # ---------------------------------------
    history = model.fit(
        train_loader,
        validation_data=valid_loader,
        epochs=CONFIG["epochs"],
        callbacks=[
            tb_callback,
            checkpoint_cb,
            WandbMetricsLogger(log_freq="epoch")
        ],
        verbose=1
    )

    model.save_weights(
        os.path.join(
            OUT_PATH,
            "weights",
            f"weights_fold{fold}-last.weights.h5"
        )
    )

    # -----------------------------------------------------
    # Evaluation
    # -----------------------------------------------------
    y_pred_prob = model.predict(valid_loader).ravel()
    y_pred = (y_pred_prob > 0.5).astype(int)
    y_true = valid_df["label"].values

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    metrics_dict = {
        "fold": fold,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred_prob),
        "aupr": average_precision_score(y_true, y_pred_prob),
        "specificity": specificity
    }

    # Log fold metrics to W&B
    wandb.log(metrics_dict)

    print(metrics_dict)
    all_metrics.append(metrics_dict)

    run.finish()

# =========================================================
# Save All Metrics
# =========================================================
metrics_df = pd.DataFrame(all_metrics)
avg_row = metrics_df.mean(numeric_only=True)
metrics_df = pd.concat(
    [metrics_df, avg_row.to_frame().T],
    ignore_index=True
)
metrics_df.loc[metrics_df.index[-1], "fold"] = "Average"

metrics_df.to_csv(
    os.path.join(
        OUT_PATH,
        f"{PROJECT_NAME}-kfold_classification_metrics.csv"
    ),
    index=False
)

print("\nAll fold metrics saved.")
metrics_df


 Fold 1 / 5


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hossainstudy7 (hossainstudy7-freelancer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


I0000 00:00:1767530705.741872   62293 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79078 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:25:00.0, compute capability: 8.0
/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/70


/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/callbacks/tensorboard.py:680: UserWarning: Model failed to serialize as JSON. Ignoring... 'LlamaTransformerDecoder' object has no attribute 'rope_scaling_factor'
  warnings.warn(f"Model failed to serialize as JSON. Ignoring... {exc}")
I0000 00:00:1767530717.504288   63281 service.cc:152] XLA service 0x7ffe200013f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767530717.504355   63281 service.cc:160]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
I0000 00:00:1767530719.273245   63281 cuda_dnn.cc:529] Loaded cuDNN version 90500
I0000 00:00:1767531017.815899   63281 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/70


/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/callbacks/tensorboard.py:680: UserWarning: Model failed to serialize as JSON. Ignoring... 'LlamaTransformerDecoder' object has no attribute 'rope_scaling_factor'
  warnings.warn(f"Model failed to serialize as JSON. Ignoring... {exc}")


95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 960ms/step - accuracy: 0.5762 - auc: 0.5986 - loss: 1.2980

95/95 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.6290 - auc: 0.6623 - loss: 0.9363 - val_accuracy: 0.7515 - val_auc: 0.8141 - val_loss: 0.5505
Epoch 2/70
95/95 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.7384 - auc: 0.8089 - loss: 0.5364 - val_accuracy: 0.7773 - val_auc: 0.8438 - val_loss: 0.5333
Epoch 3/70
95/95 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.7586 - auc: 0.8359 - loss: 0.4990 - val_accuracy: 0.7799 - val_auc: 0.8613 - val_loss: 0.5095
Epoch 4/70
95/95 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.7812 - auc: 0.8610 - loss: 0.4650 - val_accuracy: 0.7918 - val_auc: 0.8756 - val_loss: 0.4964
Epoch 5/70
95/95 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.7913 - auc: 0.8727 - loss: 0.4437 - val_accuracy: 0.8050 - val_auc: 0.8836 - val_loss: 0.4721
Epoch 6/70
95/95 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.8081 - auc: 0.8883 - loss: 0.4186 - val_accuracy: 0.7944 - val_auc: 0.8889 - val_loss: 0.4703
Epoch 7/70
95/95 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accura

In [ ]:
# =============================================================
#  EVALUATION (POST-TRAINING)
# =============================================================
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, matthews_corrcoef,
    confusion_matrix, average_precision_score
)

all_metrics = []
skf = StratifiedKFold(n_splits=CONFIG['n_splits'], shuffle=True, random_state=42)

for fold, (_, valid_idx) in enumerate(
        skf.split(df, df["Interaction"]), 1):

    print(f"\nEvaluating Fold {fold}")

    valid_df = df.iloc[valid_idx].reset_index(drop=True)

    valid_loader = DataSequenceLoader(
        valid_df,
        batch_size=CONFIG["batch_size"],
        max_len=CONFIG["max_len"],
        shuffle=False
    )

    # Rebuild model & load best weights
    model = build_model()
    model.load_weights(
        os.path.join(
            OUT_PATH, "weights",
            f"weights_fold{fold}-best.weights.h5"
        )
    )

    # Prediction
    y_prob = model.predict(valid_loader).ravel()
    y_pred = (y_prob >= 0.5).astype(int)
    y_true = valid_df["label"].values

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    metrics_dict = {
        "fold": fold,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),     # sensitivity
        "precision": precision_score(y_true, y_pred),
        "mcc": matthews_corrcoef(y_true, y_pred),
        "auc": roc_auc_score(y_true, y_prob),
        "prauc": average_precision_score(y_true, y_prob),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
        "sensitivity": tp / (tp + fn) if (tp + fn) > 0 else 0.0
    }

    print(metrics_dict)
    all_metrics.append(metrics_dict)
metrics_df = pd.DataFrame(all_metrics)

avg_row = metrics_df.mean(numeric_only=True)
avg_row["fold"] = "Average"

metrics_df = pd.concat(
    [metrics_df, avg_row.to_frame().T],
    ignore_index=True
)

metrics_df.to_csv(
    os.path.join(
        OUT_PATH,
        f"{PROJECT_NAME}-kfold_evaluation_metrics.csv"
    ),
    index=False
)

print("\nEvaluation completed and saved.")
metrics_df
